In [68]:
import pandas as pd
import nltk
import numpy as np

from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score

In [52]:
# content : 댓글
# label : 0(hate), 1(not hate)

In [43]:
data = pd.read_csv('data/hate_data.csv')
data.head()

,content,label
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다 2년전인가 좀 신선했었지 근데 이...,0
1,씨바알노무노무 술프노 오늘 저녁은 꽂등심이다ㅠㅜ,0
2,짱깨 꺼라ㅡ패쓰,0
3,그들의 사생활 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨소...,1
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1


In [44]:
train, test = train_test_split(data, test_size=0.3, random_state=42)

In [46]:
train.shape, test.shape

((12527, 2), (5369, 2))

In [47]:
train['label'].value_counts()

0    6516
1    6011
Name: label, dtype: int64

In [48]:
train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

In [50]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [14]:
content = train['content'].to_list()
label = train['label'].to_list()

train_list = list(zip(content, label))

In [16]:
okt = Okt()

In [17]:
train_docs = [(okt.morphs(sentence[0]), sentence[1]) for sentence in train_list]

In [20]:
stopwords = pd.read_csv('https://bab2min.tistory.com/attachment/cfile2.uf@241D6F475873C2B1010DEA.txt', sep='\t', header=None, names=['형태','품사','비율'])
stopwords_list = stopwords['형태'].tolist()

In [23]:
# stopword로 필요없는 문자 제거
def process_text(text) :
    clean_words = [word for word in text if word not in stopwords_list]
    return clean_words

In [24]:
train_docs = process_text(train_docs)

In [25]:
all_words = set([i for d in train_docs for i in d[0]])

In [28]:
def term_exists(doc) :
    return {word: (word in set(doc)) for word in all_words}

In [29]:
train_xy = [(term_exists(d), c) for d, c in train_docs]

In [31]:
classifier = nltk.NaiveBayesClassifier.train(train_xy)
classifier.show_most_informative_features()

# "새끼"가 들어갔을 때 0일 확률이 203.3%

Most Informative Features
                      새끼 = True                0 : 1      =    203.3 : 1.0
                      존나 = True                0 : 1      =    103.6 : 1.0
                      병신 = True                0 : 1      =     83.9 : 1.0
                      지랄 = True                0 : 1      =     59.3 : 1.0
                      깨문 = True                0 : 1      =     38.4 : 1.0
                     대가리 = True                0 : 1      =     36.6 : 1.0
                       ㅈ = True                0 : 1      =     34.7 : 1.0
                      페미 = True                0 : 1      =     31.9 : 1.0
                       쳐 = True                0 : 1      =     30.8 : 1.0
                       년 = True                0 : 1      =     26.5 : 1.0


In [54]:
test.tail()

,content,label
5364,중부고속도로 하행 진천터널 즈음 터널 진입하면 허구헌날 막힘 시발 뒤에 화물차 따라...,0
5365,주말드라마를 기다려서 보긴 거의 처음아쉬운건 2화분량을 하루에 다 하는 거ㅜㅜㅜ 저...,1
5366,응원안한다 이용진 너무 건방져 보인다 주변에 꼭 용진이처럼 근거없는 자신감으로 뭉친...,0
5367,전세계 어디도 저렇게 돈 많이 들어가서 적자보는 의료 시스템은 없음고로 이국종 또라이,0
5368,노라조 화이팅,1


In [62]:
pred = []
for sentence in test['content'] :
    pred.append(classifier.classify(term_exists(okt.morphs(sentence))))

In [78]:
print('f1 score :', np.round(f1_score(pred, test['label']), 4))
print('accuracy score :',np.round(accuracy_score(pred, test['label']), 4))
print('recall score :', np.round(recall_score(pred, test['label']), 4))

f1 score : 0.8265
accuracy score : 0.8266
recall score : 0.7669


In [80]:
content = test['content'].to_list()
label = test['label'].to_list()
test_list = list(zip(content, label))

In [81]:
test_docs = [(okt.morphs(sentence[0]), sentence[1]) for sentence in test_list]

In [82]:
test_docs = process_text(test_docs)

In [84]:
test_xy = [(term_exists(d), c) for d, c in test_docs]

In [85]:
nltk.classify.accuracy(classifier, test_xy)

0.8265971316818774